# Imports Needed:

# Data:

### Importing-

In [ ]:
Train = pd.read_csv('.csv') # Importing the csv file
Test = pd.read_csv('.csv')


## IMPORTED DATA INFORMATION 

In [ ]:
print(Train.shape)

In [ ]:
print(Train.info())

In [ ]:
print(Train.describe().T)

## Data Vizualisation:

In [ ]:
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27

# Visualising the PM 2.5 Values

sns.lineplot(data = Train["PM2.5"], color="orange", label="PM2.5")
plt.title('Visualising PM2.5 Data')
plt.show()

In [ ]:
df_corr = Train.corr()
plt.figure(figsize=(20,8))
ax = sns.heatmap( df_corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True, annot = True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
plt.show()

### SPLITTING THE DATA FOR TRAINING


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(Train.iloc[:,:8], Train.iloc[:,8], test_size=0.15, random_state=0) # This is an important function of train test split (Here train size = 90% and test size = 10%)

## Normalising

In [ ]:
cols=Train.columns

min_max_scale=MinMaxScaler()
X = min_max_scale.fit_transform(Train.iloc[:,:-1].values)

y = pd.DataFrame(conversion(Train), columns = ['PM2.5'])
Y=y.values


In [ ]:
X_t=min_max_scale.fit_transform(Test.iloc[:,:-1].values)


y_t=pd.DataFrame(conversion(Test), columns = ['PM2.5'])
Y_t=y_t.values

## APPLYING DNN

In [ ]:
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
classifier = Sequential()

classifier.add(Dense(7, activation = 'relu', input_dim = 8, kernel_regularizer= regularizers.l2(0.01)))
classifier.add(Dense(20, activation = 'relu', kernel_regularizer= regularizers.l2(0.01)))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X, Y, batch_size = 10, epochs = 100,verbose=None,shuffle=True)
prediction = classifier.predict(X_t)
acc_ANN(Y_t, prediction)

In [ ]:
plt.figure(figsize=(10, 5))
  
plt.subplot(121)
plt.title('PM 2.5 Distribution')
sns.distplot(Y)

plt.subplot(122)
plt.title('PM 2.5 Distribution for predicted values')
sns.distplot(Y_t-prediction)
plt.show()

plt.title('Scatter plot for PM 2.5 real vs predicted')
plt.scatter(Y_t,prediction)
plt.show()

plt.figure(figsize=(15,10))
plt.plot(prediction)
plt.plot(Y_t)
plt.title('Prediction vs Real PM2.5 values')
plt.ylabel('PM 2.5')
plt.xlabel('Days')
plt.legend(['Prediction', 'Real'], loc='upper left')
plt.show()

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(Y_t, prediction))
print('MSE:', metrics.mean_squared_error(Y_t, prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_t, prediction)))